<a href="https://colab.research.google.com/github/m-mazurkiewicz/PWR_MachineLearning/blob/AlexNet_master/AlexNetArchitecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D , MaxPooling2D
from keras.layers.normalization import BatchNormalization

from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

import sys
from keras.preprocessing.image import ImageDataGenerator, array_to_img, save_img
from matplotlib import pyplot as plt
import skimage.io as io
import numpy as np
import os
from tqdm import tqdm
from google.colab import drive

Wait for 8 seconds...
TensorBoard link:
http://40fb51bd.ngrok.io


In [5]:
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [0]:
num_of_classes = 4
base_dir_processed = '/content/gdrive/My Drive/PWr_AlexNet_data/processed/'
data_set = 'no-padding/resize/'

In [0]:
image_gen = ImageDataGenerator(
    featurewise_center=True,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=(0.5,1.5),
    shear_range=0.01,
    zoom_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.1
)


In [8]:
number_of_images_for_fit = 100

all_images = []
for class_name in os.listdir(base_dir_processed + data_set + 'train'):
  for image_path in tqdm(os.listdir(base_dir_processed + data_set + 'train/' + class_name)[:number_of_images_for_fit]):
    img = io.imread(base_dir_processed + data_set + 'train/' + class_name + '/' + image_path)
    all_images.append(img)
x_train = np.array(all_images)

image_gen.fit(x_train)

generator = image_gen.flow_from_directory(
    base_dir_processed + data_set + '/' + 'train',
        target_size=(227,227),
        batch_size=128,
        class_mode='categorical')

100%|██████████| 100/100 [00:00<00:00, 310.34it/s]


Found 5213 images belonging to 4 classes.


In [0]:
image_gen_test = ImageDataGenerator()
test_generator = image_gen_test.flow_from_directory(
    base_dir_processed + data_set + '/' + 'test',
        target_size=(227,227),
        batch_size=128,
        class_mode='categorical')

#1st approach

In [0]:
model = Sequential()

In [0]:
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

In [0]:
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

In [0]:
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

In [0]:
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

In [0]:
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

In [0]:
model.add(Flatten())

In [0]:
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [0]:
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [0]:
model.add(Dense(num_of_classes))
model.add(Activation('softmax'))

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
# hist = model.fit(X_tr, y_tr, validation_data = (X_val, y_val), epochs=10, callbacks=[TensorBoardColabCallback(tbc)])#000, batch_size=1000)
#To fit the model: , callbacks=[TensorBoardColabCallback(tbc)]
# hist = model.fit_generator(generator, steps_per_epoch=1, epochs=10, verbose=1, callbacks=[TensorBoardColabCallback(tbc)])
hist = model.fit_generator(generator, samples_per_epoch=10, epochs=80, verbose=1)

#3rd approach


In [3]:
# (1) Importing dependency
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)

# (2) Get Data
import tflearn.datasets.oxflower17 as oxflower17
x, y = oxflower17.load_data(one_hot=True)

# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# # 3rd Dense Layer
# model.add(Dense(1000))
# model.add(Activation('relu'))
# # Add Dropout
# model.add(Dropout(0.4))
# # Batch Normalisation
# model.add(BatchNormalization())

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

# (4) Compile
model.compile(loss='categorical_crossentropy', optimizer='adam',\
 metrics=['accuracy'])

# (5) Train
model.fit(x, y, batch_size=64, epochs=50, verbose=1, \
validation_split=0.2, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_17 (Activation)   (None, 54, 54, 96)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 26, 26, 256)       614656    
_________________________________________________________________
activation_18 (Activation)   (None, 26, 26, 256)       0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 26, 26, 256)       1024      
__________

##Test on our dataset

In [0]:
# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(227*227*3,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# # 3rd Dense Layer
# model.add(Dense(1000))
# model.add(Activation('relu'))
# # Add Dropout
# model.add(Dropout(0.4))
# # Batch Normalisation
# model.add(BatchNormalization())

# Output Layer
model.add(Dense(4))
model.add(Activation('softmax'))

model.summary()

# (4) Compile
model.compile(loss='categorical_crossentropy', optimizer='adam',\
 metrics=['accuracy'])

# (5) Train
hist = model.fit_generator(generator, samples_per_epoch=5213, epochs=50, verbose=1)
# model.fit_generator(generator, batch_size=64, epochs=50, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_33 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_34 (Activation)   (None, 27, 27, 256)       0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 27, 27, 256)       1024      
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=50, verbose=1, steps_per_epoch=40)`


Epoch 1/50
40/40 [==============================] - 95s 2s/step - loss: 1.5088 - acc: 0.4668
Epoch 2/50
40/40 [==============================] - 92s 2s/step - loss: 1.1281 - acc: 0.5501
Epoch 3/50
40/40 [==============================] - 92s 2s/step - loss: 1.0485 - acc: 0.5871
Epoch 4/50
40/40 [==============================] - 92s 2s/step - loss: 0.9287 - acc: 0.6237
Epoch 5/50
40/40 [==============================] - 92s 2s/step - loss: 0.8601 - acc: 0.6750
Epoch 6/50
40/40 [==============================] - 91s 2s/step - loss: 0.7871 - acc: 0.6978
Epoch 7/50
40/40 [==============================] - 88s 2s/step - loss: 0.7696 - acc: 0.7196
Epoch 8/50
40/40 [==============================] - 88s 2s/step - loss: 0.7238 - acc: 0.7377
Epoch 9/50
40/40 [==============================] - 87s 2s/step - loss: 0.6780 - acc: 0.7478
Epoch 10/50
40/40 [==============================] - 88s 2s/step - loss: 0.6841 - acc: 0.7496
Epoch 11/50
40/40 [==============================] - 86s 2s/step - lo